In [6]:
from CircuitErrorFiltering import CircuitErrorFiltering
import math

## Test Circuit (Inverse Quantum Fourier Transformation)

In [7]:
def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

In [12]:
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(math.pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

In [13]:
def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [14]:
def inverse_qft(circuit, n):
    """Does the inverse QFT on the first n qubits in circuit"""
    # First we create a QFT circuit of the correct size:
    qft_circ = qft(QuantumCircuit(n), n)
    # Then we take the inverse of this circuit
    invqft_circ = qft_circ.inverse()
    # And add it to the first n qubits in our existing circuit
    circuit.append(invqft_circ, circuit.qubits[:n])
    return circuit.decompose() # .decompose() allows us to see the individual gates

## Sample Code

In [15]:
from qiskit import QuantumCircuit

nqubits = 3
number = 5
qc = QuantumCircuit(nqubits)
for qubit in range(nqubits):
    qc.h(qubit)
qc.p(number*math.pi/4,0)
qc.p(number*math.pi/2,1)
qc.p(number*math.pi,2)
qc = inverse_qft(qc, nqubits)

qc.draw(fold=-1)

┌─────────┐┌─────────────┐   ┌───┐                                        
q_0: ┤ U2(0,π) ├┤ U(0,0,5π/4) ├─X─┤ H ├─■────────────────────────■─────────────
     ├─────────┤├─────────────┤ │ └───┘ │P(-π/2) ┌───┐           │             
q_1: ┤ U2(0,π) ├┤ U(0,0,5π/2) ├─┼───────■────────┤ H ├─■─────────┼─────────────
     ├─────────┤└┬───────────┬┘ │                └───┘ │P(-π/2)  │P(-π/4) ┌───┐
q_2: ┤ U2(0,π) ├─┤ U(0,0,5π) ├──X──────────────────────■─────────■────────┤ H ├
     └─────────┘ └───────────┘                                            └───┘

In [109]:
transpile(circuits = qc, basis_gates = ['cx', 'id', 'rz', 'sx', 'x']).draw()

global phase: 4.9087
     ┌─────────┐┌────┐┌──────────┐     ┌───┐     ┌─────────┐┌────┐┌─────────┐»
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(-π/4) ├──■──┤ X ├──■──┤ Rz(π/2) ├┤ √X ├┤ Rz(π/4) ├»
     ├─────────┤├────┤└┬────────┬┘  │  └─┬─┘  │  └─────────┘└────┘└─────────┘»
q_1: ┤ Rz(π/2) ├┤ √X ├─┤ Rz(-π) ├───┼────┼────┼──────────────────────────────»
     ├─────────┤├────┤┌┴────────┴┐┌─┴─┐  │  ┌─┴─┐                            »
q_2: ┤ Rz(π/2) ├┤ √X ├┤ Rz(-π/2) ├┤ X ├──■──┤ X ├────────────────────────────»
     └─────────┘└────┘└──────────┘└───┘     └───┘                            »
«                          ┌──────────┐                                      »
«q_0: ──■───────────────■──┤ Rz(-π/8) ├──────────────────────────────────────»
«     ┌─┴─┐┌─────────┐┌─┴─┐├─────────┬┘┌────┐┌─────────┐                     »
«q_1: ┤ X ├┤ Rz(π/4) ├┤ X ├┤ Rz(π/4) ├─┤ √X ├┤ Rz(π/4) ├──■───────────────■──»
«     └───┘└─────────┘└───┘└─────────┘ └────┘└─────────┘┌─┴─┐┌─────────┐┌─┴─┐»
«q_2: ──────────────────────────────────────────────────┤ X ├┤ Rz(π/4) ├┤ X ├»
«                                                       └───┘└─────────┘└───┘»
«                                                                   
«q_0: ──────────────■───────────────■───────────────────────────────
«                   │               │                               
«q_1: ──────────────┼───────────────┼───────────────────────────────
«     ┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐┌────┐┌─────────┐
«q_2: ┤ Rz(-π/4) ├┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(3π/8) ├┤ √X ├┤ Rz(π/2) ├
«     └──────────┘└───┘└─────────┘└───┘└──────────┘└────┘└─────────┘

In [21]:
from qiskit import transpile
from qiskit_aer import AerSimulator

# Transpile with given basis gate set
qc_transpiled = transpile(circuits = qc, basis_gates = ['cx', 'id', 'rz', 'sx', 'x'])
qc_transpiled.measure_all()

# Create noisy simulator backend
sim = AerSimulator()

# Transpile circuit for noisy basis gates
circ = transpile(qc_transpiled, sim)

# Run and get counts
result = sim.run(circ, seed_simulator=9, shots=1000).result()
result.get_counts()

{'101': 1000}

## Sample Code with Error Filtering

In [27]:
# Error filtering code
EF_qc = CircuitErrorFiltering(qc, 0)

In [29]:
# Quantum circuit with error filtering
EF_qc.SingleQubitBitFlip().draw(fold=-1)

┌─────────┐              ┌────┐        ┌──────────┐     ┌───┐          ┌─────────┐     ┌────┐     ┌─────────┐                     ┌──────────┐                                                                                                               ░ ┌─┐            
   q_0: ──■──┤ Rz(π/2) ├────────■─────┤ √X ├─────■──┤ Rz(-π/4) ├──■──┤ X ├───────■──┤ Rz(π/2) ├──■──┤ √X ├──■──┤ Rz(π/4) ├──■───────────────■──┤ Rz(-π/8) ├────────────────────────────────────────────────────■───────────────■────────────────■─────────────────────■───░─┤M├────────────
          │  ├─────────┤┌────┐  │   ┌─┴────┴─┐   │  └──────────┘  │  └─┬─┘       │  └─────────┘  │  └────┘  │  └─────────┘┌─┴─┐┌─────────┐┌─┴─┐├─────────┬┘┌────┐┌─────────┐                                   │               │                │                     │   ░ └╥┘┌─┐         
   q_1: ──┼──┤ Rz(π/2) ├┤ √X ├──┼───┤ Rz(-π) ├───┼────────────────┼────┼─────────┼───────────────┼──────────┼─────────────┤ X ├┤ Rz(π/4) ├┤ X ├┤ Rz(π/4) ├─┤ √X ├┤ Rz(π/4) ├──■───────────────■────────────────┼───────────────┼────────────────┼─────────────────────┼───░──╫─┤M├─────────
          │  ├─────────┤├────┤  │  ┌┴────────┴┐  │              ┌─┴─┐  │       ┌─┴─┐             │          │             └───┘└─────────┘└───┘└─────────┘ └────┘└─────────┘┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐  │  ┌────┐┌─────────┐  │   ░  ║ └╥┘┌─┐      
   q_2: ──┼──┤ Rz(π/2) ├┤ √X ├──┼──┤ Rz(-π/2) ├──┼──────────────┤ X ├──■────■──┤ X ├─────────────┼──────────┼───────────────────────────────────────────────────────────────┤ X ├┤ Rz(π/4) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(3π/8) ├──┼──┤ √X ├┤ Rz(π/2) ├──┼───░──╫──╫─┤M├──────
        ┌─┴─┐└──┬───┬──┘└────┘┌─┴─┐└──────────┘┌─┴─┐   ┌───┐    └───┘     ┌─┴─┐├───┤           ┌─┴─┐      ┌─┴─┐   ┌───┐   ┌───┐                                             └───┘└─────────┘└───┘└──────────┘└───┘└─────────┘└───┘└──────────┘  │  └────┘└─────────┘┌─┴─┐ ░  ║  ║ └╥┘┌─┐   
   q_3: ┤ X ├───┤ I ├─────────┤ X ├────────────┤ X ├───┤ I ├──────────────┤ X ├┤ I ├───────────┤ X ├──────┤ X ├───┤ I ├───┤ I ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────■──────────────┤ X ├─░──╫──╫──╫─┤M├───
        └───┘   └───┘         └───┘            └───┘   └───┘              └───┘└───┘           └───┘      └───┘   └───┘   └───┘                                                                                                               ┌─┴─┐┌─┴─┐            └───┘ ░  ║  ║  ║ └╥┘┌─┐
   q_4: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ X ├──────────────────░──╫──╫──╫──╫─┤M├
                                                                                                                                                                                                                                              └───┘└───┘                  ░  ║  ║  ║  ║ └╥┘
meas: 5/═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩══╩═
                                                                                                                                                                                                                                                                             0  1  2  3  4

In [30]:
# Apply single qubit bit flip code
EF_qc.PostProcessing()

Probability of post-selection: 1.0
Post processed counts: {'000': 0, '001': 0, '010': 0, '011': 0, '100': 0, '101': 1000, '110': 0, '111': 0}


{'000': 0,
 '001': 0,
 '010': 0,
 '011': 0,
 '100': 0,
 '101': 1000,
 '110': 0,
 '111': 0}